In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-5/train.csv',parse_dates=['Date'])
df_test=pd.read_csv('/kaggle/input/covid19-global-forecasting-week-5/test.csv',parse_dates=['Date'])

In [ ]:
# 展示训练样本
print(df_train.shape)
df_train.head()

In [ ]:
# 测试数据
print(df_test.shape)
df_test.head()

In [ ]:
# 合并测试与训练数据集
df_test.rename(columns={'ForecastId':'Id'}, inplace=True)
df_test['Id'] = df_test['Id'].astype('str')+'a'
df = pd.concat([df_train, df_test], axis =0).copy()
df

### Feature Generation

In [ ]:
df.Date = pd.to_datetime(df.Date)
df['Day']=df['Date'].dt.day
df['Month']=df['Date'].dt.month
df['Day of the week']=df['Date'].dt.dayofweek
df['isweekend']=np.where(df['Day of the week'].isin([5,6]),1,0)

In [ ]:
df

In [ ]:
# 将标签进行转换成0-1变量
from sklearn.preprocessing import LabelEncoder
Target_label_encoder=LabelEncoder()
df['Target_label']=Target_label_encoder.fit_transform(df.Target)

In [ ]:
df[df['Id'].str.contains('a').isnull()]

In [ ]:
X=df[df['Id'].str.contains('a').isnull()][['Population', 'Weight', 'Day', 'Month', 'Day of the week', 'isweekend', 'Target_label']]
X

In [ ]:
y=df[df['Id'].str.contains('a').isnull()]['TargetValue']
y

In [ ]:
# 缩小将y<0的值删掉
X=X.loc[y[y>=0].index]
y=y[y>=0]

### Modeling

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier #Decision Tree
model = DecisionTreeClassifier()
model.fit(X_train,y_train)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))



In [ ]:
# 预测，结果为op
df_test_fin=df[df['Id'].str.contains('a').notnull()][['Population', 'Weight', 'Day', 'Month', 'Day of the week', 'isweekend', 'Target_label']]
op = model.predict(df_test_fin)
df_sub=pd.DataFrame(df_test.index+1,columns=['Id'])
df_sub['TargetValue'] = op

In [ ]:
a=df_sub.groupby(['Id'])['TargetValue'].quantile(q=0.05).reset_index()
b=df_sub.groupby(['Id'])['TargetValue'].quantile(q=0.5).reset_index()
c=df_sub.groupby(['Id'])['TargetValue'].quantile(q=0.95).reset_index()


a.columns=['Id','q0.05']
b.columns=['Id','q0.5']
c.columns=['Id','q0.95']
a=pd.concat([a,b['q0.5'],c['q0.95']],1)

In [ ]:
sub=pd.melt(a, id_vars=['Id'], value_vars=['q0.05','q0.5','q0.95'])
sub['variable']=sub['variable'].str.replace("q","", regex=False)
sub['ForecastId_Quantile']=sub['Id'].astype(str)+'_'+sub['variable']
sub['TargetValue']=sub['value']
sub=sub[['ForecastId_Quantile','TargetValue']]
sub.reset_index(drop=True,inplace=True)
sub.head()

In [ ]:
sub.to_csv("submission.csv",index=False)